# Working with xAI Grok in Pixeltable

Pixeltable's xAI integration enables you to use Grok models for chat completions and image generation via the xAI API.

### Prerequisites
- An xAI account with an API key (https://console.x.ai/)

### Important notes

- xAI API usage incurs costs based on your xAI plan
- The xAI API is OpenAI-compatible, using the same SDK
- Be mindful of sensitive data and consider security measures when integrating with external services


First you'll need to install the OpenAI SDK (xAI uses an OpenAI-compatible API) and set up your xAI API key.


In [ ]:
# For local development, use the local pixeltable path
import sys
sys.path.insert(0, '/Users/pierre/pixeltable')


In [ ]:
import os
import getpass

if 'XAI_API_KEY' not in os.environ:
    os.environ['XAI_API_KEY'] = getpass.getpass('Enter your xAI API key: ')


Now let's create a Pixeltable directory to hold the tables for our demo.


In [ ]:
import pixeltable as pxt

# Remove the 'xai_demo' directory and its contents, if it exists
pxt.drop_dir('xai_demo', force=True)
pxt.create_dir('xai_demo')


## Chat Completions with Grok

Grok is xAI's flagship language model, designed to provide truthful, insightful answers. Let's use it for chat completions.


from pixeltable.functions import xai

# Create a table for chat conversations
chat_t = pxt.create_table('xai_demo.chat', {'prompt': pxt.String})

# Create message format for Grok
messages = [
    {'role': 'system', 'content': 'You are Grok, a helpful AI assistant created by xAI.'},
    {'role': 'user', 'content': chat_t.prompt}
]

# Add computed column for chat completions
chat_t.add_computed_column(
    response=xai.chat_completions(messages=messages, model='grok-3')
)

# Extract just the response text
chat_t.add_computed_column(
    answer=chat_t.response['choices'][0]['message']['content']
)


In [ ]:
# Insert some prompts
chat_t.insert([
    {'prompt': 'What is the meaning of life, the universe, and everything?'},
    {'prompt': 'Explain quantum computing in simple terms.'},
    {'prompt': 'What makes a good cup of coffee?'},
])


In [ ]:
# View the results
chat_t.select(chat_t.prompt, chat_t.answer).collect()


## Image Generation

xAI also offers image generation capabilities with Grok models. Let's generate some images from text prompts.


In [ ]:
# Create a table for image generation
img_t = pxt.create_table('xai_demo.images', {'prompt': pxt.String})

# Add computed column for image generation
img_t.add_computed_column(
    generated_image=xai.image_generations(img_t.prompt, model='grok-2-image')
)


In [ ]:
# Insert image prompts
img_t.insert([
    {'prompt': 'A futuristic cityscape at sunset with flying cars'},
    {'prompt': 'A cute robot helping in a garden'},
])


In [ ]:
# View the generated images
img_t.select(img_t.prompt, img_t.generated_image).collect()


## Available Models

### Language Models (Chat Completions)

| Model | Description |
|-------|-------------|
| `grok-4` | Latest and most capable Grok 4 model |
| `grok-4-fast` | Faster Grok 4 variant |
| `grok-3` | Grok 3 model (default) |
| `grok-3-fast` | Faster Grok 3 variant |
| `grok-2-1212` | Grok 2 model |
| `grok-2-vision-1212` | Grok 2 with vision capabilities |

### Image Generation Models

| Model | Description |
|-------|-------------|
| `grok-2-image` | Image generation model |

### Model Parameters

The `chat_completions` function supports additional parameters via `model_kwargs`:

- `max_tokens`: Maximum tokens in the response
- `temperature`: Sampling temperature (0-2)
- `top_p`: Nucleus sampling parameter
- `frequency_penalty`: Penalty for repeated tokens
- `presence_penalty`: Penalty for new topics
- `stop`: Stop sequences


## Learn More

- [xAI Documentation](https://docs.x.ai/)
- [xAI Console](https://console.x.ai/)
- [Grok Models](https://docs.x.ai/docs/models)
- [API Reference](https://docs.x.ai/docs/api-reference)
